In [1]:
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms
import torch
from torch import nn
from torchviz import make_dot

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to a fixed size
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [3]:
path = 'Dataset/melanomaCancerImageDataset/train'
testPath = 'Dataset/melanomaCancerImageDataset/test'

trainData = datasets.ImageFolder(root = path, transform = transform)
testData = datasets.ImageFolder(root = testPath, transform = transform)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device : {device}')

Using device : cuda


In [5]:
batchSize = 16

trainLoader = DataLoader(trainData, batch_size = batchSize, shuffle = True, num_workers = 4)
testLoader = DataLoader(testData, batch_size = batchSize, shuffle = True, num_workers = 4)

In [6]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.stride = stride

        # Downsample layer if the number of channels changes
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        identity = self.downsample(identity)

        out += identity
        out = self.relu(out)
        return out

class ModifiedVGG(nn.Module):
    def __init__(self, num_classes=1000):
        super(ModifiedVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(64, 64),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(128, 128),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(256, 256),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            BasicBlock(512, 512)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Conv2d(512, 4096, kernel_size=7),
            nn.ReLU(inplace=True),
            nn.Conv2d(4096, 4096, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(4096, num_classes, kernel_size=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1) # Flatten
        return x

# Example usage:
model = ModifiedVGG(num_classes=2)  # Example with 2 output classes
loss_fun = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.to(device)


ModifiedVGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential()
    )
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_s

In [7]:
model.eval()
outputs = 0
for inputs, labels in trainLoader:
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    break
make_dot(outputs, params=dict(list(model.named_parameters()))).render("torchviz_ModifiedVGG", format="png")


'torchviz_ModifiedVGG.png'

In [8]:
epochs = 10
best_accuracy = 0
best_epoch = 0
for epoch in range(epochs):
    model.train()
    for inputs, labels in trainLoader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fun(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Training - Epoch {epoch+1}/{epochs}, Loss: {loss.item()}') 
    
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testLoader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            outputs = model(inputs)
            loss = loss_fun(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    average_test_loss = test_loss / len(testLoader)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = epoch

        # Save the trained model if needed
        torch.save(model.state_dict(), 'vgg_resnet_model_melanoma_cancer.pth')

    print(f'Testing - Epoch {epoch+1}/{epochs}, Loss: {average_test_loss}, Accuracy: {accuracy}')

Training - Epoch 1/10, Loss: 0.40754270553588867
Testing - Epoch 1/10, Loss: 0.6980584952831268, Accuracy: 0.683
Training - Epoch 2/10, Loss: 0.4256214201450348
Testing - Epoch 2/10, Loss: 0.46924086356163025, Accuracy: 0.785
Training - Epoch 3/10, Loss: 0.32585272192955017
Testing - Epoch 3/10, Loss: 0.4884646935462952, Accuracy: 0.758
Training - Epoch 4/10, Loss: 0.19179187715053558
Testing - Epoch 4/10, Loss: 0.44648976099491117, Accuracy: 0.8
Training - Epoch 5/10, Loss: 0.23862186074256897
Testing - Epoch 5/10, Loss: 0.4137549493312836, Accuracy: 0.8135
Training - Epoch 6/10, Loss: 0.798897922039032
Testing - Epoch 6/10, Loss: 0.5587660797834396, Accuracy: 0.7805
Training - Epoch 7/10, Loss: 0.9001162648200989
Testing - Epoch 7/10, Loss: 0.5539620269536972, Accuracy: 0.717
Training - Epoch 8/10, Loss: 0.3317115902900696
Testing - Epoch 8/10, Loss: 0.41341561675071714, Accuracy: 0.814
Training - Epoch 9/10, Loss: 0.9002116322517395
Testing - Epoch 9/10, Loss: 0.3735980795621872, Ac